Необходимые импорты

In [2]:
import requests
from bs4 import BeautifulSoup
import time

In [9]:
url = 'https://habr.com/ru/search/'

keyword_list = ['python', 'анализ данных', 'deepseek']


def get_queries(url, query_list):
    query_result_list = []
    for query in query_list:
        params = {
            'q': query,
            'target_types': 'posts',
            'order': 'relevance'
        }

        res = requests.get(url, params)
        time.sleep(0.2)
        soup = BeautifulSoup(res.text)
        query_result_list.append(soup)

        # top_level_tag = soup.find('div', class_='tm-pagination__pages')

        # nested_tags = top_level_tag.find_all(
        #     'div', class_='tm-pagination__page-group')
        # last_nested_tag = nested_tags[-1] if nested_tags else None
        # last_page = last_nested_tag.find_all(
        #     'a', class_='tm-pagination__page')[-1].text

        # print(last_page)
    return query_result_list

def get_query_result(query):
    article_list = query.find_all('article', class_='tm-articles-list__item')
    return article_list

query_list = get_queries(url, keyword_list)

for item in query_list:
    print(len(get_query_result(item)))
    



20
20
20
